In [1]:
import requests
import numpy as np
import pandas as pd

# Data as of May 25, 2024 15:52 PM UTC+7
# get heroes data
heroes_url = 'https://api.opendota.com/api/heroes'
resp_hero = requests.get(heroes_url)
resp_hero_json = resp_hero.json()

heroes_df = pd.DataFrame(resp_hero_json)
heroes_df.head()

,id,name,localized_name,primary_attr,attack_type,roles,legs
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
1,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Carry]",2
2,3,npc_dota_hero_bane,Bane,all,Ranged,"[Support, Disabler, Nuker, Durable]",4
3,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Nuker, Initiator]",2
4,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker]",2


In [2]:
# get unique roles
heroes_df_mod = heroes_df.copy()
all_roles = set(role for role_list in heroes_df['roles'] for role in role_list)
all_roles

for role in all_roles:
    heroes_df_mod['role_' + role.lower()] = heroes_df_mod['roles'].apply(lambda x: 1 if role in x else 0)

heroes_df_mod = heroes_df_mod.drop(columns=['roles'])
heroes_df_mod.head()

,id,name,localized_name,primary_attr,attack_type,legs,role_escape,role_pusher,role_disabler,role_initiator,role_durable,role_carry,role_support,role_nuker
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,2,1,0,0,0,0,1,0,1
1,2,npc_dota_hero_axe,Axe,str,Melee,2,0,0,1,1,1,1,0,0
2,3,npc_dota_hero_bane,Bane,all,Ranged,4,0,0,1,0,1,0,1,1
3,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,2,0,0,1,1,0,1,0,1
4,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,2,0,0,1,0,0,0,1,1


In [7]:
display(heroes_df_mod.info(),
        heroes_df_mod.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              124 non-null    int64 
 1   name            124 non-null    object
 2   localized_name  124 non-null    object
 3   primary_attr    124 non-null    object
 4   attack_type     124 non-null    object
 5   legs            124 non-null    int64 
 6   role_escape     124 non-null    int64 
 7   role_pusher     124 non-null    int64 
 8   role_disabler   124 non-null    int64 
 9   role_initiator  124 non-null    int64 
 10  role_durable    124 non-null    int64 
 11  role_carry      124 non-null    int64 
 12  role_support    124 non-null    int64 
 13  role_nuker      124 non-null    int64 
dtypes: int64(10), object(4)
memory usage: 13.7+ KB


None

,id,legs,role_escape,role_pusher,role_disabler,role_initiator,role_durable,role_carry,role_support,role_nuker
count,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000
mean,64.056452,2.064516,0.411290,0.233871,0.741935,0.443548,0.395161,0.572581,0.370968,0.709677
std,37.476779,1.166901,0.494064,0.425008,0.439345,0.498818,0.490869,0.496711,0.485024,0.455753
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.750000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,63.500000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
75%,94.250000,2.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,138.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
# get all unique hero id
hero_id_list = heroes_df_mod['id'].unique()

# initialize empty dictionary
hero_game_duration = {}

# counter
i = 1

for hero_id in hero_id_list:
    # get the hero game durations from the api
    heroes_duration_url = f'https://api.opendota.com/api/heroes/{hero_id}/durations'
    resp_hero_duration = requests.get(heroes_duration_url)
    resp_hero_duration_json = resp_hero_duration.json()
    
    # append to initialized dictionary
    hero_game_duration[hero_id] = resp_hero_duration_json
    print(f'No. {i} successful retrieval for hero id: {hero_id}')
    i += 1

# initialize empty list to group all hero game duration dataframes
hero_game_duration_dfs = []

for hero_id, durations in hero_game_duration.items():
    temp_df = pd.DataFrame(durations)
    temp_df['hero_id'] = hero_id
    hero_game_duration_dfs.append(temp_df)

hero_game_duration_df = pd.concat(hero_game_duration_dfs, ignore_index=True)
hero_game_duration_df.hero_id.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 119, 120, 121, 123,
       126, 128, 129, 135, 136, 137, 138])

In [25]:
hero_game_duration_df.head()

,duration_bin,games_played,wins,hero_id
0,5700,1,0,1
1,3300,285,147,1
2,7500,1,0,1
3,1800,2332,1162,1
4,3900,48,26,1


In [29]:
# get hero stats data
hero_stats_url = 'https://api.opendota.com/api/heroStats'
resp_hero_stats = requests.get(hero_stats_url)
resp_hero_stats_json = resp_hero_stats.json()

resp_hero_stats_json[0]
hero_stats_df = pd.DataFrame(resp_hero_stats_json)
hero_stats_df_mod = hero_stats_df.copy()

# drop these columns because some are already in heroes_df_mod
# and trend columns are not described in the doc so i have no idea what it represents.
cols_to_drop = ['name', 'roles', 'primary_attr', 'attack_type', 'img',
                'icon', 'turbo_picks_trend', 'turbo_wins_trend',
                'pub_pick_trend', 'pub_win_trend']
hero_stats_df_mod = hero_stats_df_mod.drop(columns=cols_to_drop, axis=1)
hero_stats_df_mod.head()

,id,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_mr,base_attack_min,base_attack_max,base_str,...,7_win,8_pick,8_win,turbo_picks,turbo_wins,pro_pick,pro_win,pro_ban,pub_pick,pub_win
0,1,120,0.75,75,0.0,1,25,29,33,19,...,13732,13198,6519,241581,123901,150,71,283,494559,250815
1,2,120,2.50,75,0.0,0,25,30,34,25,...,20406,16200,8660,373879,210043,281,156,309,664033,366899
2,3,120,0.25,75,0.0,1,25,-1,5,23,...,8276,12375,6425,104164,50573,221,106,316,192845,96812
3,4,120,0.25,75,0.0,2,25,35,41,24,...,4476,3494,1758,166841,80502,95,51,107,185626,93449
4,5,120,0.25,75,0.0,0,25,30,36,17,...,11910,11541,5884,270125,140765,268,141,59,453308,235028


In [99]:
# get hero matchup data
hero_id_lists = heroes_df_mod['id'].unique()

batch_size = int(len(hero_id_lists) / 6) # fetch 6 heroes per batch

hero_matchup_dict = {}

def process_batch(hero_id_lists):
    # get data from api
    for hero_id in hero_id_lists:
        heroes_matchup_url = f'https://api.opendota.com/api/heroes/{hero_id}/matchups'
        resp_hero_matchup = requests.get(heroes_matchup_url)
        if resp_hero_matchup.status_code == 200:
            resp_hero_matchup_json = resp_hero_matchup.json()

            # Append the data to the dictionary
            hero_matchup_dict[hero_id] = resp_hero_matchup_json
            print(f'Successfully retrieved data for hero ID: {hero_id}')
        else:
            print(f"Failed to fetch data for hero ID: {hero_id}")

# get data in batches
for i in range(0, len(hero_id_lists), batch_size):
    batch = hero_id_lists[i:i + batch_size]
    process_batch(batch)
    
hero_matchup_dfs = []

for hero_id, matchup in hero_matchup_dict.items():
    temp_df = pd.DataFrame(matchup)
    temp_df['matchup_hero_id'] = hero_id
    hero_matchup_dfs.append(temp_df)

hero_matchup_df = pd.concat(hero_matchup_dfs, ignore_index=True)
hero_matchup_df.matchup_hero_id.unique()

Successfully retrieved data for hero ID: 1
Successfully retrieved data for hero ID: 2
Successfully retrieved data for hero ID: 3
Successfully retrieved data for hero ID: 4
Successfully retrieved data for hero ID: 5
Successfully retrieved data for hero ID: 6
Successfully retrieved data for hero ID: 7
Successfully retrieved data for hero ID: 8
Successfully retrieved data for hero ID: 9
Successfully retrieved data for hero ID: 10
Successfully retrieved data for hero ID: 11
Successfully retrieved data for hero ID: 12
Successfully retrieved data for hero ID: 13
Successfully retrieved data for hero ID: 14
Successfully retrieved data for hero ID: 15
Successfully retrieved data for hero ID: 16
Successfully retrieved data for hero ID: 17
Successfully retrieved data for hero ID: 18
Successfully retrieved data for hero ID: 19
Successfully retrieved data for hero ID: 20
Successfully retrieved data for hero ID: 21
Successfully retrieved data for hero ID: 22
Successfully retrieved data for hero ID: 

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 119, 120, 121, 123,
       126, 128, 129, 135, 136, 137, 138])

In [101]:
hero_matchup_df.head()

,hero_id,games_played,wins,matchup_hero_id
0,10,205,98,1
1,94,193,92,1
2,86,187,88,1
3,5,136,59,1
4,71,131,58,1


In [102]:
dir_path = f'/Users/lpt814/Desktop/data-science-projects/dota/'
# save data to csv
'''
heroes_df_mod.to_csv(dir_path + 'heroes_may_25_2024.csv')
hero_game_duration_df.to_csv(dir_path + 'hero_game_duration_may_25_2024.csv')
hero_stats_df_mod.to_csv(dir_path + 'hero_stats_may_25_2024.csv')
hero_matchup_df.to_csv(dir_path + 'hero_matchup_may_25_2024.csv')
'''